In [114]:
#verilerin yüklenmesi
veriler=pd.read_csv("/content/maaslar.csv")
veriler

,unvan,Egitim Seviyesi,maas
0,Cayci,1,2250
1,Sekreter,2,2500
2,Uzman Yardimcisi,3,3000
3,Uzman,4,4000
4,Proje Yoneticisi,5,5500
5,Sef,6,7500
6,Mudur,7,10000
7,Direktor,8,15000
8,C-level,9,25000
9,CEO,10,50000


In [116]:
#KORELASYON İLİŞKİLERİ
corr_matrix = veriler.corr()
print(corr_matrix)
def en_yuksek_iliskiler(corr_matrix):
    # Korelasyon matrisini sütunlara göre sıralama
    sorted_corr = corr_matrix.unstack().sort_values(ascending=False)

    # Sıralanmış korelasyon değerlerinin en yüksek olduğu sütun ikililerini yazdırma
    for i, (s1, s2) in enumerate(sorted_corr.index):
        if s1 != s2:  # Aynı sütun ikilisini tekrarlamamak için
            print(f"{s1}-{s2}: {sorted_corr[i]}")

# Örnek kullanım
en_yuksek_iliskiler(corr_matrix)


Egitim Seviyesi-maas: 0.8179494074776198
maas-Egitim Seviyesi: 0.8179494074776198


In [117]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

def one_hot_encode(data, columns):
    # Kategorik sütunları seçme
    categorical_data = data[columns]

    # One-Hot Encoder oluşturma
    encoder = OneHotEncoder(sparse=False, drop='first')

    # Kategorik veriyi dönüştürme
    encoded_data = encoder.fit_transform(categorical_data)

    # One-Hot Encoding sonucunu DataFrame'e dönüştürme
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns))

    # Dönüştürülmüş veriyi orijinal veriyle birleştirme
    preprocessed_data = pd.concat([data.drop(columns, axis=1), encoded_df], axis=1)

    return preprocessed_data




def min_max_scale(data, column):
    # Min-Max Scaler oluşturma
    scaler = MinMaxScaler()

    # Sütunu seçme ve ölçeklendirme
    scaled_column = scaler.fit_transform(data[[column]])

    # Ölçeklendirilmiş sütunu yeni bir DataFrame'e dönüştürme
    scaled_data = pd.DataFrame(scaled_column, columns=[column])

    # Ölçeklendirilmiş sütunu orijinal veri setine ekleme
    data[column] = scaled_data[column]

    return data


def fill_nan_with_mean(data):
    for column in data.columns:
        if data[column].isnull().any():
            mean = data[column].mean()
            data[column].fillna(mean, inplace=True)
    return data


def veriOnIsleme(data, columnsForOneHotEncoder, columnsForMinMaxScaler):
    dataForOneHot = one_hot_encode(data, columnsForOneHotEncoder)
    for column in columnsForMinMaxScaler:
        dataForOneHot = min_max_scale(dataForOneHot, column)  # Düzeltme: dataForOneHot kullanılmalı
    result = fill_nan_with_mean(dataForOneHot)
    return result

In [124]:
veriSetim=pd.read_csv("/content/maaslar.csv")
#veriOnİslemeliSetDuzelt=veriOnIsleme(veriSetim, ["Surname", "Geography", "Gender"], ["CreditScore", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"])
#veriOnİslemeliSetDuzelt=veriOnİslemeliSetDuzelt.drop(["RowNumber","CustomerId"], axis=1)

veriOnİslemeliSetDuzelt=veriOnIsleme(veriSetim,[], ["Egitim Seviyesi","maas"])

MyX = veriOnİslemeliSetDuzelt["Egitim Seviyesi"]
MyY = veriOnİslemeliSetDuzelt["maas"]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [125]:
MyX

0    0.000000
1    0.111111
2    0.222222
3    0.333333
4    0.444444
5    0.555556
6    0.666667
7    0.777778
8    0.888889
9    1.000000
Name: Egitim Seviyesi, dtype: float64

In [126]:
MyY

0    0.000000
1    0.005236
2    0.015707
3    0.036649
4    0.068063
5    0.109948
6    0.162304
7    0.267016
8    0.476440
9    1.000000
Name: maas, dtype: float64

In [127]:
def parcalamaIslemi(A,B):
  from sklearn.model_selection import train_test_split
  X_train, X_test, Y_train, Y_test = train_test_split(A, B,test_size=0.2,random_state=42)
  return X_train, X_test, Y_train, Y_test

In [128]:
MyX_train, MyX_test, MyY_train, MyY_test=parcalamaIslemi(MyX.values, MyY.values.reshape(-1, 1))

Linear **Regression**

In [177]:
#Linear Regression+
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(MyX_train.reshape(-1, 1), MyY_train)
tahminCoklu = lin_reg.predict(MyX_test.reshape(-1, 1))

In [ ]:
#LINEAR REGRESSION METRİCS
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Calculate metrics
mse = mean_squared_error(MyY_test, tahminCoklu)
r2 = r2_score(MyY_test, tahminCoklu)

# Plot the predicted values against the actual values
plt.scatter(MyY_test, tahminCoklu)
plt.plot([MyY_test.min(), MyY_test.max()], [MyY_test.min(), MyY_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.show()

# Plot the residuals
residuals = MyY_test - tahminCoklu
plt.scatter(tahminCoklu, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals Plot')
plt.show()

# Print the metrics
print("Mean Squared Error (MSE):", mse)
print("Coefficient of Determination (R^2):", r2)

#   MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(MyY_test, tahminCoklu)
print("Ortalama Mutlak Hata (MAE):", mae)

# RMSE
rmse = np.sqrt(mse)
print("Kök Ortalama Kare Hata (RMSE):", rmse)



#plt.scatter(MyX_train.reshape(-1, 1), MyY_train)
#plt.plot(MyX_test.reshape(-1, 1),lin_reg.predict(MyX_test.reshape(-1, 1)))
#plt.show()


POlİNOMAL **REGRESSİON**

In [179]:
#POlİNOMAL REGRESSİON+
from sklearn.preprocessing import PolynomialFeatures
poly_reg=PolynomialFeatures(degree=4)
x_poly=poly_reg.fit_transform(MyX_train.reshape(-1, 1))
lin2_reg=LinearRegression()
lin2_reg.fit(x_poly,MyY_train)
tahminPolynomial=lin2_reg.predict(x_poly)

In [ ]:
#POLIYNOMAİL REGRESSION METRİCS
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Veri setini train ve test kümelerine ayırma
MyX_train, MyX_test, MyY_train, MyY_test = parcalamaIslemi(MyX.values, MyY.values.reshape(-1, 1))

# Polinom özelliklerini oluşturma
poly_reg = PolynomialFeatures(degree=2)
x_poly_train = poly_reg.fit_transform(MyX_train.reshape(-1, 1))

# Polinom regresyon modelini oluşturma ve eğitme
lin2_reg = LinearRegression()
lin2_reg.fit(x_poly_train, MyY_train)

# Test veri kümesi üzerinde tahmin yapma
x_poly_test = poly_reg.transform(MyX_test.reshape(-1, 1))
tahminPolynomial = lin2_reg.predict(x_poly_test)

# Hata metriklerini hesaplama
mse = mean_squared_error(MyY_test, tahminPolynomial)
r2 = r2_score(MyY_test, tahminPolynomial)

# Tahminleri görselleştirme
plt.scatter(MyX_test, MyY_test, color='red')
plt.plot(MyX_test, tahminPolynomial, color='blue')
plt.title('Maaş Tahminleri')
plt.xlabel('Yıl Deneyimi')
plt.ylabel('Maaş')
plt.show()

# Gerçek ve tahmin edilen değerler arasındaki farkı görselleştirme
residuals = MyY_test - tahminPolynomial
plt.scatter(tahminPolynomial, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Artık (Residuals)')
plt.title('Artık Dağılımı')
plt.show()

# Hata metriklerini yazdırma
print("Ortalama Kare Hatası (MSE):", mse)
print("Determinasyon Katsayısı (R^2):", r2)

#   MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(MyY_test, tahminPolynomial)
print("Ortalama Mutlak Hata (MAE):", mae)

# RMSE
rmse = np.sqrt(mse)
print("Kök Ortalama Kare Hata (RMSE):", rmse)

Support Vektor **Regression**

In [183]:
#Support Vektor Regression+
from sklearn.svm import SVR
svr_reg = SVR(kernel='sigmoid')
svr_reg.fit(MyX_train.reshape(-1, 1), MyY_train)
tahminSVR=svr_reg.predict(MyX_test.reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#SUPPORT VECTOR REGRESSION METRİCS
mse_svr = mean_squared_error(MyY_test, tahminSVR)
r2_svr = r2_score(MyY_test, tahminSVR)

# Tahminleri görselleştirme (SVR)
plt.scatter(MyX_test, MyY_test, color='red')
plt.plot(MyX_test, tahminSVR, color='blue')
plt.title('Maaş Tahminleri (SVR)')
plt.xlabel('Yıl Deneyimi')
plt.ylabel('Maaş')
plt.show()

# Gerçek ve tahmin edilen değerler arasındaki farkı görselleştirme (SVR)
residuals_svr = MyY_test.reshape(-1) - tahminSVR
plt.scatter(tahminSVR, residuals_svr)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Tahmin Edilen (SVR)')
plt.ylabel('Artık (Residuals)')
plt.title('Artık Dağılımı (SVR)')
plt.show()


# Linear Regression metriklerini yazdırma
print("Linear Regression Metrics:")
print("Mean Squared Error (MSE):", mse)
print("Coefficient of Determination (R^2):", r2)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Support Vector Regression metriklerini yazdırma
print("Support Vector Regression Metrics:")
print("Mean Squared Error (MSE):", mse_svr)
print("Coefficient of Determination (R^2):", r2_svr)

Decision **Tree**

In [159]:
#Decision Tree+
from sklearn.tree import DecisionTreeRegressor
r_dt=DecisionTreeRegressor(random_state=(42))
r_dt.fit(MyX_train.reshape(-1, 1), MyY_train)
tahminDesicionTree=r_dt.predict(MyX_test.reshape(-1, 1))

In [ ]:
# Decision Tree METRİCS


# Calculate metrics
mse_dt = mean_squared_error(MyY_test, tahminDesicionTree)
r2_dt = r2_score(MyY_test, tahminDesicionTree)

# Plot the predicted values against the actual values
plt.scatter(MyY_test, tahminDesicionTree)
plt.plot([MyY_test.min(), MyY_test.max()], [MyY_test.min(), MyY_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted (Decision Tree)')
plt.show()

# Plot the residuals
residuals_dt = MyY_test.reshape(-1) - tahminDesicionTree
plt.scatter(tahminDesicionTree, residuals_dt)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals Plot (Decision Tree)')
plt.show()

# Print the metrics
print("Decision Tree Metrics:")
print("Mean Squared Error (MSE):", mse_dt)
print("Coefficient of Determination (R^2):", r2_dt)
print("Ortalama Mutlak Hata (MAE):", mean_absolute_error(MyY_test, tahminDesicionTree))
print("Kök Ortalama Kare Hata (RMSE):", np.sqrt(mse_dt))

Random **Forest**

In [185]:
#Random Forest+
from sklearn.ensemble import RandomForestRegressor
rf_reg=RandomForestRegressor(n_estimators=20,random_state=(42))
rf_reg.fit(MyX_train.reshape(-1, 1), MyY_train)
tahminRandomForest=rf_reg.predict(MyX_test.reshape(-1, 1))

<ipython-input-185-0ff78582da95>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_reg.fit(MyX_train.reshape(-1, 1), MyY_train)


In [ ]:
#RANDOM FOREST METRİCS

# Calculate metrics
mse_rf = mean_squared_error(MyY_test, tahminRandomForest)
r2_rf = r2_score(MyY_test, tahminRandomForest)

# Plot the predicted values against the actual values
plt.scatter(MyY_test, tahminRandomForest)
plt.plot([MyY_test.min(), MyY_test.max()], [MyY_test.min(), MyY_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted (Random Forest)')
plt.show()

# Plot the residuals
residuals_rf = MyY_test.reshape(-1) - tahminRandomForest
plt.scatter(tahminRandomForest, residuals_rf)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals Plot (Random Forest)')
plt.show()

# Print the metrics
print("Mean Squared Error (MSE):", mse_rf)
print("Coefficient of Determination (R^2):", r2_rf)

# MAE
mae_rf = mean_absolute_error(MyY_test, tahminRandomForest)
print("Mean Absolute Error (MAE):", mae_rf)

# RMSE
rmse_rf = np.sqrt(mse_rf)
print("Root Mean Squared Error (RMSE):", rmse_rf)
